# Generalized Disjunctive Programming in Pyomo <img src="Pyomo-GDP-150.png" width="20%" style="float: right; margin: 1rem;" />

<br style="clear:right;" />

Pyomo.GDP provides support for Generalized Disjunctive Programming<sup>[1](#ref-1)</sup> within the Pyomo algebraic modeling language.
GDP is an extension of Disjunctive Programming<sup>[2](#ref-2)</sup> from the operations research community to include nonlinear relationships.
The classic form for a GDP is:

$$
\begin{aligned}
\min obj = &\ f(x, z) \\
s.t. \quad &\ Ax+Bz \leq d\\
&\ g(x,z) \leq 0\\
&\ \bigvee_{i\in D_k} \left[
    \begin{gathered}
    Y_{ik} \\
    M_{ik} x + N_{ik} z \leq e_{ik} \\
    r_{ik}(x,z)\leq 0\\
    \end{gathered}
\right] \quad k \in K\\
&\ \Omega(Y) = True \\
&\ x \in X \subseteq \mathbb{R}^n\\
&\ Y \in \{True, False\}^{p}\\
&\ z \in Z \subseteq \mathbb{Z}^m\\
\end{aligned} \tag{GDP}
$$

Here, we have the minimization of an objective $obj$ subject to global linear constraints $Ax+Bz \leq d$ and nonlinear constraints $g(x,z) \leq 0$, with conditional linear constraints $M_{ik} x + N_{ik} z \leq e_{ik}$ and nonlinear constraints $r_{ik}(x,z)\leq 0$.
These conditional constraints are collected into disjuncts $D_k$, organized into disjunctions $K$. Finally, there are logical propositions $\Omega(Y) = True$.
Decision/state variables can be continuous $x$, Boolean $Y$, and/or integer $z$.

In [1]:
# Required python imports
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.core.expr.logical_expr import *
from pyomo.core.plugins.transform.logical_to_linear import update_boolean_vars_from_binary
m = ConcreteModel()
m.my_set = RangeSet(4)

## Disjunct

Disjuncts represent groupings of relational expressions (e.g. algebraic constraints) summarized by a Boolean indicator variable $Y$ through implication:

$$
\left.
\begin{aligned}
& Y_{ik} \Rightarrow & M_{ik} x + N_{ik} z &\leq e_{ik}\\
& Y_{ik} \Rightarrow & r_{ik}(x,z) &\leq 0
\end{aligned}
\right.\qquad \forall i \in D_k, \forall k \in K
$$

Logically, this means that if $Y_{ik} = True$, then the constraints $M_{ik} x + N_{ik} z \leq e_{ik}$ and $r_{ik}(x,z) \leq 0$ must be satisfied.
However, if $Y_{ik} = False$, then the corresponding constraints are ignored.
Note that $Y_{ik} = False$ does **not** imply that the corresponding constraints are *violated*.

## Disjunction

Disjunctions describe a logical *or* relationship between two or more Disjuncts.

## BooleanVar

For historical reasons, the `indicator_var` variable automatically added to disjuncts in `Pyomo.GDP` has type `binary` rather than `Boolean`.
As a result, we define the `BooleanVar` object in Pyomo to represent Boolean variables, analogous to `Var` for numeric variables.
`BooleanVar` can be indexed over a Pyomo `Set`, as below:

In [2]:
m.Y = BooleanVar(m.my_set)
m.Y.display()

Y : Size=4, Index=my_set
    Key : Value : Fixed : Stale
      1 :  None : False :  True
      2 :  None : False :  True
      3 :  None : False :  True
      4 :  None : False :  True


## LogicalStatement

Likewise, logical propositions in `Pyomo.GDP` are expressed using `LogicalStatment` objects, analogous to numeric `Constraint` objects.

In [3]:
m.p = LogicalStatement(expr=m.Y[1].implies(m.Y[2] & m.Y[3]) | m.Y[4])
m.p.pprint()

p : Size=1, Index=None, Active=True
    Key  : Body                         : Active
    None : Y[1] >> (Y[2] & Y[3]) | Y[4] :   True


# Demonstrations

1. [Supported Expressions](./supported-expressions.ipynb)
2. [Logical Propositions with Disjunctions](./use-with-disjunctions.ipynb)
3. [Use with Pyomo.Network](./use-with-network.ipynb)  <-- *Work in progress*
4. [Eight Process Problem](./demo-8pp.ipynb)
5. [Strip Packing Problem](./demo-strip-packing.ipynb)

# References

1. Trespalacios \& Grossmann, 2014: https://doi.org/10.1002/cite.201400037
2. Balas, 2018: https://doi.org/10.1007/978-3-030-00148-3